# This workflow demonstrates the workflow to perform land cover classification using Sentinel-2 Imagery from Microsoft Planetary Computer and pre-trained deep learning model from ArcGIS Online

The workflow includes the following steps:
1. Find all the Sentinel-2 scenes from 2022 that covers Los Angeles
2. Create a RasterCollection from all the scenes
3. Sort the RasterCollection by cloud cover to get a scene with the least cloud
4. Use the Classify Pixels Using DeepLearning tool to classify the image and save output

##### Note: It's highly recommended to use a NVIDIA GPU-enabled machine to perform this workflow. You may need to install the driver manually (https://learn.microsoft.com/en-us/azure/virtual-machines/windows/n-series-driver-setup)
##### The output may not appear correct if you use GPU fobut have not installed the driver

#### Estimated running time: 30 minutes (with GPU) / 4.5 hours (without GPU)

### Import relevant modules and log in to ArcGIS Online

In [ ]:
import arcpy
from arcpy import AIO
import getpass
from arcgis.gis import GIS

In [ ]:
# Log in to ArcGIS online to access deep learning models
portal="https://arcgis.com"
username = getpass.getpass('Enter AGOL username:')
password = getpass.getpass('Enter AGOL password: ')
params = arcpy.SignInToPortal(portal,username,password)
gis = GIS(portal)

### Create a RasterCollection of potential scenes from planetary computer archive 

In [ ]:
# Create an AIO object from the cloud storage connection file for data access
a = AIO(r'C:\AMPC_Resources\ACS_Files\esrims_pc_sentinel-2-l2a.acs')

In [ ]:
# Define the query to search for images
query = {
    "collections": ["sentinel-2-l2a"], # Sentinel-2 level 2A product (https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
    "bbox": [-118.5006071, 33.9690070, -118.4480544, 34.0457182], # Define bounding box over Los Angeles
    "datetime": "2022-01-01/2022-12-31", # Define time range
    "limit": 1000, # Define max number of results to be returned
          }

In [ ]:
# Create a RasterCollection object that contains the search results
rc = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1",
                                  query=query,
                                  attribute_dict={
                                      "Name":"id",
                                      "Cloud Cover":"eo:cloud_cover",
                                      "StdTime":"datetime",
                                      "Spatial Reference":"proj:epsg",
                                      "Extent": "bbox",
                                    })

### Find image with least cloud cover from RasterCollection for classification

In [ ]:
# Sort the collection by ascending cloud cover percentage and select the image with least cloud
rc_sort_by_cloud_cover = rc.sort('Cloud Cover')
input_raster = rc_sort_by_cloud_cover[0]['Raster']

In [ ]:
# Visualize the sorted RasterCollection
rc_sort_by_cloud_cover

In [ ]:
# Choose the the first raster from the sorted colleciton as input raster
input_raster = rc_sort_by_cloud_cover[0]['Raster']

### Perform land cover classification using a deep learning model from ArcGIS Online

Now we need to use ArcGIS API for Python to search for a suitable deep learning model (dlpk) on ArcGIS Online
<br>
Documentation on how to perform a search on ArcGIS Online:
https://developers.arcgis.com/python/guide/accessing-and-creating-content/#searching-for-content-in-arcgis-online

In [ ]:
# Search for a suitable deep learning model on ArcGIS Online
query = 'sentinel land cover classification dlpk, owner:esri_analytics'
classification_models = gis.content.search(query = query, max_items = 30, outside_org = True)
for model in classification_models:
    display(model)

Here we want to use **Land Cover Classification (Sentinel-2)** model.
<br>
The model link is https://www.arcgis.com/home/item.html?id=afd124844ba84da69c2c533d4af10a58.
<br>
We need to obtain the model id which is **afd124844ba84da69c2c533d4af10a58**

In [ ]:
deeplearning_model_id = 'afd124844ba84da69c2c533d4af10a58'

In [ ]:
# Classify the image using a deep learning model
# Make sure you have installed the GPU driver if you specify processorType="GPU",  otherwise specify processorType="CPU"
with arcpy.EnvManager(processorType="GPU"): 
    out_classified_raster = arcpy.ia.ClassifyPixelsUsingDeepLearning(
        in_raster = input_raster,
        in_model_definition = "https://www.arcgis.com/sharing/rest/content/items/" + deeplearning_model_id,
        arguments  ="padding 128;batch_size 4;predict_background True;test_time_augmentation True;tile_size 512;output_label_level 2;sentinel_imagery_level 2;merge_classes true",
        processing_mode = "PROCESS_AS_MOSAICKED_IMAGE",
        out_classified_folder = None,
        out_featureclass = None,
        overwrite_attachments = "NO_OVERWRITE"
    )
    # Save output to a local file
    out_classified_raster.save(r"C:\Temp\LA_classified.tif")

In [ ]:
# Display the output
out_classified_raster